#CherryPick prototype
Scrape a page for DOIS, clean and check them, then get some metadata from the crossRef API

In [147]:
import re
import scrapy
from scrapy.http import TextResponse
import requests
import json
#regex modified from http://stackoverflow.com/questions/27910/finding-a-doi-in-a-document-or-page
#Alix Axel's regex, with modifications http://stackoverflow.com/users/89771/alix-axel
#found on stackoverflow
doi_re = re.compile(r'\b(10[.][0-9]{3,}(?:[.][0-9]+)*/(?:(?!["&\'()])\S)+)')
target = 'http://www.cardiff.ac.uk/chemistry/research/publications/2014-publications'
r = requests.get(target)
response=TextResponse(r.url,body=r.text, encoding='utf-8')

#all_dois = doi_re.findall(response.xpath('string(.)').extract()[0])
all_dois = doi_re.findall(r.text)
all_dois = [clean_doi(d) for d in all_dois]
all_dois = list(set(all_dois)) #uniqify
print all_dois
api_stub = 'http://api.crossref.org/works/'
reqs = [api_stub+d for d in all_dois]
fails = 0
for req in reqs:
    #print(req)
    r = requests.get(req)
    resp=TextResponse(r.url,body=r.text, encoding='utf-8')
    try:
        jsonresponse = json.loads(resp.body_as_unicode())''
        t=jsonresponse["message"]['title']
        print(t)
    except:
        fails+=1
print(str(fails) + ' failed dois from ' + str(len(reqs)) + ' requests')
print('failure rate: ' + str(float(fails)/len(reqs)))



[u'10.1021/ic501411w', u'10.1038/ncomms4332', u'10.1039/c4cy00027g', u'10.1016/j.tet.2014.02.043', u'10.1016/j.jcat.2013.04.005', u'10.1039/C4OB01916D', u'10.1039/C4CE00308J', u'10.1063/1.4903961', u'10.1002/cphc.201402271', u'10.1021/op500221s', u'10.1021/bi500507v', u'10.1021/bi500238q', u'10.1002/chem.201304005', u'10.1063/1.4889780', u'10.1002/anie.201308997', u'10.1016/j.ccr.2014.02.003', u'10.1039/C3SM52877D', u'10.1016/j.jorganchem.2013.08.031', u'10.1039/c4ta02521k', u'10.1021/bi500508z', u'10.1007/s11244-013-0229-5', u'10.1002/chem.201303736', u'10.1021/ja5066366', u'10.1107/S1600536814019035', u'10.1039/C4MD00265B', u'10.1021/om500847j', u'10.1002/chem.201302053', u'10.1002/9781118695708.ch1', u'10.1021/ic403011h', u'10.1039/C4SC00545G', u'10.1039/c4cy00387j', u'10.1039/c3dt52972j', u'10.1039/c4gc00087k', u'10.1016/j.tet.2014.03.078', u'10.1556/JFC-D-13-00030', u'10.1039/C3SC52745J', u'10.1039/c4dt00519h', u'10.1007/s11243-013-9789-2', u'10.1016/B978-0-08-097742-3.00821-1', u

In [153]:
d =u'10.1002/9781118<d>778173</a>,'
clean_doi(d)

u'10.1002/9781118<d>778173'

In [145]:
def clean_doi(d):
    if d[-1] in'.,':
        d=d[:-1]
    ###strip trailing html tags
    clean = re.sub(r'<[^>]+>$','',d)
    return clean

In [150]:
target = 'http://www.cardiff.ac.uk/chemistry/research/publications/2014-publications'
r = requests.get(target)
response=TextResponse(r.url,body=r.text, encoding='utf-8')
target_space = '//body/div/div/div/div[2]/div[2]'
target = response.xpath(target_space).xpath('string(.)').extract()[0] 

In [152]:
len(target)

1